In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

Making the kaggle.json file secured

In [3]:
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c playground-series-s3e14

playground-series-s3e14.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip playground-series-s3e14.zip

Archive:  playground-series-s3e14.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
from sklearn.ensemble import BaggingRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
import warnings

In [7]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv',index_col=0)

In [8]:
train.columns

Index(['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia',
       'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange',
       'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange',
       'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds',
       'yield'],
      dtype='object')

In [9]:
X = train.drop('yield', axis=1)
y = train['yield']

In [10]:
lr = LinearRegression()
kfold = KFold(n_splits=5,shuffle=True,random_state=24)
bagg = BaggingRegressor(estimator=lr, random_state=24)
params = {'n_estimators':[10,20,50]}
gcv = GridSearchCV(bagg,param_grid=params,cv=kfold,scoring='r2')
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=BaggingRegressor(estimator=LinearRegression(),
                                        random_state=24),
             param_grid={'n_estimators': [10, 20, 50]}, scoring='r2')

In [11]:
gcv.best_params_,gcv.best_score_

({'n_estimators': 20}, -1.298099634122499e+18)

In [12]:
scaler_std = StandardScaler()
scaler_minmax = MinMaxScaler()
scaler = scaler_std
lr = LinearRegression()
bagg = BaggingRegressor(estimator=lr, random_state=24)
pipe = Pipeline([('SCL',None),('BAGG',bagg)])
params = {'BAGG__n_estimators':[10,20,50],
          'SCL':[None, scaler_std, scaler_minmax ]}
gcv = GridSearchCV(pipe,param_grid=params,cv=kfold,scoring='r2')
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('SCL', None),
                                       ('BAGG',
                                        BaggingRegressor(estimator=LinearRegression(),
                                                         random_state=24))]),
             param_grid={'BAGG__n_estimators': [10, 20, 50],
                         'SCL': [None, StandardScaler(), MinMaxScaler()]},
             scoring='r2')

In [13]:
gcv.best_params_,gcv.best_score_

({'BAGG__n_estimators': 20, 'SCL': StandardScaler()}, 0.7940922837177092)

In [14]:
best_model = gcv.best_estimator_
y_pred = best_model.predict(test)
y_pred[y_pred<0]

array([], dtype=float64)

In [ ]:
submit = pd.read_csv('sample_submission.csv')
submit['yield'] = y_pred
submit.to_csv('sbt_bagg_scl_lr.csv',index=False)

In [ ]:
#!kaggle competitions submit -c playground-series-s3e14 -f 'sbt_bagg_scl_lr.csv' -m "{'BAGG__n_estimators': 20, 'SCL': StandardScaler()}"

In [15]:
!kaggle competitions submissions playground-series-s3e14

fileName             date                 description                                          status    publicScore  privateScore  
-------------------  -------------------  ---------------------------------------------------  --------  -----------  ------------  
sbt_bagg_scl_lr.csv  2024-11-09 09:29:26  {'BAGG__n_estimators': 20, 'SCL': StandardScaler()}  complete  368.68694    363.65289     
